# Sistema RAG con PDF

Este notebook te guiará a través de la construcción de un sistema RAG (Retrieval-Augmented Generation) usando un documento PDF.

## 1. Importar librerías necesarias

En esta sección importamos todas las librerías que utilizaremos para el procesamiento del PDF, la creación de embeddings, la base de datos vectorial y la interacción con el modelo de lenguaje.

In [ ]:
import os
import logging
import ipywidgets as widgets
from IPython.display import display, Markdown
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## 2. Configurar constantes y logging

Definimos las rutas de archivos, los nombres de los modelos y configuramos el sistema de logging para poder ver información relevante durante la ejecución del notebook.

In [ ]:
DOC_PATH = "../data/2312.10997v5.pdf"  # Ruta al PDF a analizar
MODEL_NAME = "llama3.2"      # Nombre del modelo de lenguaje
EMBEDDING_MODEL = "nomic-embed-text"  # Modelo de embeddings
VECTOR_STORE_NAME = "rag_simple"      # Nombre de la colección vectorial

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## 3. Cargar y visualizar el PDF

En este paso cargamos el documento PDF y mostramos un resumen o las primeras líneas para que puedas ver el contenido que se va a procesar.

In [ ]:
def ingest_pdf(doc_path):
    """Cargar documentos PDF."""
    if os.path.exists(doc_path):
        loader = UnstructuredPDFLoader(file_path=doc_path)
        data = loader.load()
        logging.info("PDF cargado correctamente.")
        return data
    else:
        logging.error(f"No se encontró el archivo PDF en la ruta: {doc_path}")
        return None

In [ ]:
data = ingest_pdf(DOC_PATH)

## 4. Dividir el documento en fragmentos

Dividimos el texto del PDF en fragmentos más pequeños (chunks) para facilitar la búsqueda y el procesamiento posterior. Mostramos algunos ejemplos de estos fragmentos.

In [ ]:
def split_documents(documents):
    """Divide documentos en fragmentos más pequeños."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=300)
    chunks = text_splitter.split_documents(documents)
    logging.info(f"Documentos divididos en {len(chunks)} fragmentos.")
    return chunks

In [ ]:
chunks = split_documents(data)

## 5. Crear la base de datos vectorial

Convertimos los fragmentos en vectores numéricos (embeddings) y los almacenamos en una base de datos vectorial. Esto permite realizar búsquedas semánticas eficientes sobre el contenido del PDF.

In [ ]:
def create_vector_db(chunks):
    """Crear una base de datos vectorial a partir de fragmentos de documentos."""
    vector_db = Chroma.from_documents(
        documents=chunks,
        embedding=OllamaEmbeddings(model=EMBEDDING_MODEL),
        collection_name=VECTOR_STORE_NAME,
        persist_directory="../chroma_db",
    )
    logging.info("Base de datos vectorial creada.")
    return vector_db

In [ ]:
vector_db = create_vector_db(chunks)

## 6. Inicializar el modelo de lenguaje

Cargamos el modelo de lenguaje que se usará para generar respuestas y para crear queries alternativas.

In [ ]:
llm = ChatOllama(model=MODEL_NAME)

## 7. Crear el recuperador de información (retriever)

Configuramos el recuperador multi-query, que genera varias versiones de una pregunta para mejorar la recuperación de información relevante del PDF.

In [ ]:
def create_retriever(vector_db, llm):
    """Crear un recuperador básico."""
    # Usamos el retriever básico configurado para devolver los 5 documentos más relevantes
    retriever = vector_db.as_retriever(search_kwargs={"k": 5})
    logging.info("Retriever creado.")
    return retriever

In [ ]:
retriever = create_retriever(vector_db, llm)

## 8. Construir la cadena RAG

Enlazamos el recuperador, el modelo de lenguaje y el parser de salida en una cadena que implementa el flujo RAG completo.

In [ ]:
def create_chain(retriever, llm):
    """Crear la cadena"""
    template = """Responde la pregunta basándote ÚNICAMENTE en el siguiente contexto:\n{context}\nPregunta: {question}\n"""

    prompt = ChatPromptTemplate.from_template(template)

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    logging.info("Cadena RAG creada exitosamente.")
    return chain

In [ ]:
chain = create_chain(retriever, llm)

## 9. Realizar una consulta de ejemplo y mostrar la respuesta

Ejecutamos una consulta de ejemplo sobre el sistema RAG y mostramos la respuesta generada, explicando cada paso del proceso.

In [ ]:
pregunta = "¿Que es RAG?"
print(f"\nConsulta de ejemplo: {pregunta}\n")
respuesta = chain.invoke(input=pregunta)
print("Respuesta generada por el sistema:")
display(Markdown(respuesta))

## 10. Interfaz interactiva para consultas

Le damos una vuelta a nuestro codigo para hacerlo interactivo. En esta celda te permite preguntar de manera interactiva utilizando widgets.

In [ ]:
pregunta = input("Introduce tu pregunta: ")
respuesta = chain.invoke(input=pregunta)

out = widgets.Output()
with out:
    display(Markdown(respuesta))

display(out)

## 11. Función principal interactiva

Otra mejora es definir una función principal (`main`) que permita ejecutar toda la cadena RAG y realizar preguntas sobre su contenido utilizando widgets.

In [ ]:
def main():
    data = ingest_pdf(DOC_PATH)

    chunks = split_documents(data)

    vector_db = create_vector_db(chunks)

    llm = ChatOllama(model=MODEL_NAME)

    retriever = create_retriever(vector_db, llm)

    chain = create_chain(retriever, llm)
    
    pregunta = input("Introduce tu pregunta: ")
    respuesta = chain.invoke(input=pregunta)

    out = widgets.Output()
    with out:
        display(Markdown(respuesta))

    display(out)
    
main()

## 11. Interfaz interactiva avanzada

En esta última celda se implementa una interfaz interactiva mejorada que permite seleccionar dinámicamente el archivo PDF y realizar preguntas sobre su contenido. Utiliza widgets para facilitar la selección del documento y la introducción de consultas, mostrando las respuestas generadas por el sistema RAG de manera clara y accesible.

In [ ]:
def main():
    data_dir = "../data"
    pdf_files = [f for f in os.listdir(data_dir) if f.lower().endswith(".pdf")]
    
    if not pdf_files:
        print("No se encontraron archivos PDF en la carpeta ../data")
        return

    file_selector = widgets.Dropdown(
        options=pdf_files,
        description='PDF:',
        disabled=False,
    )
    pregunta_box = widgets.Text(
        value='',
        placeholder='Introduce tu pregunta',
        description='Pregunta:',
        disabled=False
    )
    ejecutar_btn = widgets.Button(
        description="Ejecutar",
        button_style='success'
    )
    output = widgets.Output()

    display(file_selector, pregunta_box, ejecutar_btn, output)

    def on_ejecutar_clicked(b):
        output.clear_output()
        with output:
            selected_path = os.path.join(data_dir, file_selector.value)
            data = ingest_pdf(selected_path)
            if data is None:
                print("No se pudo cargar el PDF.")
                return
            chunks = split_documents(data)
            vector_db = create_vector_db(chunks)
            llm = ChatOllama(model=MODEL_NAME)
            retriever = create_retriever(vector_db, llm)
            chain = create_chain(retriever, llm)
            pregunta = pregunta_box.value
            if not pregunta.strip():
                print("Por favor, introduce una pregunta.")
                return
            respuesta = chain.invoke(input=pregunta)
            display(Markdown(respuesta))

    ejecutar_btn.on_click(on_ejecutar_clicked)
    
main()

## 12. Interfaz interactiva con síntesis de voz

En esta celda, la interfaz interactiva permite seleccionar un archivo PDF, realizar preguntas sobre su contenido y escuchar la respuesta generada mediante síntesis de voz utilizando ElevenLabs.

In [ ]:
def main():
    data_dir = "../data"
    pdf_files = [f for f in os.listdir(data_dir) if f.lower().endswith(".pdf")]
    
    if not pdf_files:
        print("No se encontraron archivos PDF en la carpeta ../data")
        return

    file_selector = widgets.Dropdown(
        options=pdf_files,
        description='PDF:',
        disabled=False,
    )
    pregunta_box = widgets.Text(
        value='',
        placeholder='Introduce tu pregunta',
        description='Pregunta:',
        disabled=False
    )
    ejecutar_btn = widgets.Button(
        description="Ejecutar",
        button_style='success'
    )
    output = widgets.Output()

    display(file_selector, pregunta_box, ejecutar_btn, output)

    def on_ejecutar_clicked(b):
        output.clear_output()
        with output:
            selected_path = os.path.join(data_dir, file_selector.value)
            data = ingest_pdf(selected_path)
            if data is None:
                print("No se pudo cargar el PDF.")
                return
            chunks = split_documents(data)
            vector_db = create_vector_db(chunks)
            llm = ChatOllama(model=MODEL_NAME)
            retriever = create_retriever(vector_db, llm)
            chain = create_chain(retriever, llm)
            pregunta = pregunta_box.value
            if not pregunta.strip():
                print("Por favor, introduce una pregunta.")
                return
            respuesta = chain.invoke(input=pregunta)
            display(Markdown(respuesta))
            
            # === TEXT TO SPEECH ===
            from elevenlabs.client import ElevenLabs
            from elevenlabs import stream
            from dotenv import load_dotenv

            load_dotenv()
            api_key = os.getenv("ELEVENLABS_API_KEY")
            if api_key:
                client = ElevenLabs(api_key=api_key)
                audio_stream = client.text_to_speech.stream(
                    text=respuesta, 
                    voice_id="JBFqnCBsd6RMkjVDRZzb",
                    model_id="eleven_multilingual_v2"
                )
                stream(audio_stream)
            else:
                print("No se encontró la API KEY de ElevenLabs.")

    ejecutar_btn.on_click(on_ejecutar_clicked)

main()